In [3]:
"""
Created on Wed Jun 28 04:04:25 2023

@author: hugo511
"""

import argparse
import pandas as pd
# pd.set_option('display.float_format', lambda x:'%.2f'%x)
import numpy as np
np.set_printoptions(suppress=True)
import time
import warnings
warnings.filterwarnings('ignore')
import os 
import statsmodels.api as sm

parser = argparse.ArgumentParser(description="Asset Pricing - Tail stocks return regression")
parser.add_argument('--dataset_start', type=int, default=196301, help='dataset start date')
parser.add_argument('--dataset_end', type=int, default=201012, help='dataset end date')
parser.add_argument('--qt', type=float, default=0.05, help='quantile value - u_t')
parser.add_argument('--save_dir', type=str, default='../testTailStocksRET_table2/', help='results save dirs')
parser.add_argument('--isTest', type=bool, default=True, help='is Test or not')
parser.add_argument('--Lwindow', type=int, default=1, help='length of rolling window')
parser.add_argument('-f')
args = parser.parse_args()

def cal_aggHorizon_ret(Yret_init, H):
    Yagg = np.array([])
    for idx in range(1, Yret_init.shape[0] - H + 1):
        Yagg_idx = (Yret_init[idx: idx + H] + 1).prod() - 1
        Yagg = np.append(Yagg, Yagg_idx)
    return Yagg

class computeHodrick1992VCovMatrix:
    def __init__(self, Ysub_H, Ysub_1, X, H):
        self.Ysub_H = Ysub_H
        self.Ysub_1 = Ysub_1
        self.X = X
        self.H = H 
        self.nrows, self.ncols = X.shape
        C = np.ones([Ysub_1.shape[0], 1])
        rg_sub_1 = sm.OLS(Ysub_1, C).fit()
        self.resid_sub_1 = rg_sub_1.resid
        
    def computeZ(self):
        Z = np.zeros([self.ncols, self.ncols])
        for i in range(self.nrows):
            Xi = np.expand_dims(self.X[i,], axis=1)
            Z = Z + Xi @ Xi.T
        Z = (1/self.nrows) * Z
        return Z
    
    def computelittleM(self, i, H):
        m = np.zeros([self.ncols, 1])
        for h in range(H): # h = 0:H-1
            Xi = np.expand_dims(self.X[i-h], axis=1)
            m = m + Xi
        m = self.resid_sub_1[i] * m
        return m
    
    def computeS(self, H):
        S = np.zeros([self.ncols, self.ncols])
        for i in range(H-1, self.nrows):
            S = S + (self.computelittleM(i, H) @ (self.computelittleM(i, H)).T)
        S = (1/self.nrows) * S
        return S
    
    def forward(self, H):
        Ssub_H = self.computeS(H)
        Zsub_H = self.computeZ()
        Vsub_H = np.linalg.inv(Zsub_H) @ Ssub_H @ np.linalg.inv(Zsub_H) / self.nrows
        return Vsub_H


if __name__ == '__main__':
    # dataset
    args.dataset_start = 196301
    args.dataset_end = 202212
    args.isTest = False
    args.tail_dir = './logs/Hill_estimate_results/'
    # tail lambda_t
    tail_estimates = pd.read_csv(os.path.join(args.tail_dir, f'Hillestimate_plotresults_{args.dataset_start}_{args.dataset_end}.csv'))
    tail_estimates = tail_estimates[(tail_estimates['date'] >= args.dataset_start) & (tail_estimates['date'] <= args.dataset_end)]

    # market return vwretd monthly
    mkt_monthlyret = pd.read_csv('./Dataset/MarketRET_Monthly_192601_202212.csv')
    mkt_monthlyret = mkt_monthlyret[(mkt_monthlyret['date'] >= args.dataset_start) & (mkt_monthlyret['date'] <= args.dataset_end)].reset_index(drop=True)
    
    # equity_predictor 
    equity_predictor = pd.read_csv('./Dataset/EquityPredictorDataset_Monthly_192601_202212.csv')
    equity_predictor = equity_predictor[(equity_predictor['date'] >= args.dataset_start) & (equity_predictor['date'] <= args.dataset_end)].reset_index(drop=True)

    # fitting dataset
    fitdataset = pd.concat([equity_predictor['date'], mkt_monthlyret['vwretd'], tail_estimates['lambda_t'], equity_predictor.iloc[:, 1:-2]], axis=1)
    fitdataset = fitdataset.rename(columns={'lambda_t':'Tail'})
    fitdataset.columns
    for col in fitdataset.columns[2:]:
        fitdataset[col] = (fitdataset[col] - fitdataset[col].mean()) / fitdataset[col].std()

In [5]:
#%% table 2
regressors = ['Book-to-market', 'Default return spread', 'Default yield spread', 'Dividend payout ratio', 'Dividend price ratio'
              , 'Earnings price ratio', 'Inflation', 'Long-term return', 'Long-term yield', 'Net equity expansion', 'Stock volatility'
              , 'Term spread', 'Treasury-bill rate']
# 1M 
args.Lwindow = 1
fitdataset1M = pd.DataFrame()
fitdataset1M['date'] = fitdataset['date'][:-args.Lwindow]
fitdataset1M['vwretd'] = fitdataset['vwretd'].shift(-1).dropna()   
fitdataset1M['Tail'] = fitdataset['Tail'][:-args.Lwindow]
for col in regressors:
    fitdataset1M[col] = fitdataset[col][:-args.Lwindow]
TailStocksRET1M_df = pd.DataFrame(columns=['Rz.', 'Tail Coeff.', 'Tail t-stat.', 'Coeff.', 't-stat', 'R2'])
for regressor in regressors:
    fitY = fitdataset1M['vwretd']
    fitX = pd.concat([fitdataset1M['Tail'], fitdataset1M[regressor]], axis=1)
    fitX = sm.add_constant(fitX)
    rgmodel = sm.OLS(fitY, fitX).fit()
    coeff_Tail, coeff_regressor = np.round(rgmodel.params['Tail'] * 12 * 100, 2), np.round(rgmodel.params[f'{regressor}'] * 12 * 100, 2)
    tstat_Tail, tstat_regressor = np.round(rgmodel.tvalues['Tail'], 2), np.round(rgmodel.tvalues[f'{regressor}'], 2)
    rsquared = np.round(rgmodel.rsquared * 100, 1)
    # Hodrick standard error
    Ysub_H = np.array(fitY); Ysub_1 = np.array(fitY); X = np.array(fitX); H = args.Lwindow
    HodrickV = computeHodrick1992VCovMatrix(Ysub_H, Ysub_1, X, H)
    stdHH = np.sqrt(np.diag(HodrickV.forward(H))) 
    tHH_Tail, tHH_regressor = np.round((rgmodel.params / stdHH)['Tail'], 2), np.round((rgmodel.params / stdHH)[f'{regressor}'], 2)
    TailStocksRET1M_df = TailStocksRET1M_df.append({'Rz.':f'{regressor}', 'Tail Coeff.':coeff_Tail, 'Tail t-stat.':tstat_Tail, 'Coeff.':coeff_regressor, 't-stat':tstat_regressor, 'R2':rsquared},ignore_index=True)
print(TailStocksRET1M_df)

# 1Y 
args.Lwindow = 12
fitdataset1Y = pd.DataFrame()
fitdataset1Y['date'] = fitdataset['date'][:-args.Lwindow]
fitdataset1Y['vwretd'] = cal_aggHorizon_ret(fitdataset['vwretd'], args.Lwindow)   
fitdataset1Y['Tail'] = fitdataset['Tail'][:-args.Lwindow]
for col in regressors:
    fitdataset1Y[col] = fitdataset[col][:-args.Lwindow]
TailStocksRET1Y_df = pd.DataFrame(columns=['Rz.', 'Tail Coeff.', 'Tail t-stat.', 'Coeff.', 't-stat', 'R2'])
for regressor in regressors:
    fitY = fitdataset1Y['vwretd']
    fitX = pd.concat([fitdataset1Y['Tail'], fitdataset1Y[regressor]], axis=1)
    fitX = sm.add_constant(fitX)
    rgmodel = sm.OLS(fitY, fitX).fit()
    coeff_Tail, coeff_regressor = np.round(rgmodel.params['Tail'] * 100, 2), np.round(rgmodel.params[f'{regressor}'] * 100, 2)
    tstat_Tail, tstat_regressor = np.round(rgmodel.tvalues['Tail'], 2), np.round(rgmodel.tvalues[f'{regressor}'], 2)
    rsquared = np.round(rgmodel.rsquared * 100, 1)
    # Hodrick standard error
    Ysub_H = np.array(fitY); Ysub_1 = np.array(fitdataset1M['vwretd']); X = np.array(fitX); H = args.Lwindow 
    HodrickV = computeHodrick1992VCovMatrix(Ysub_H, Ysub_1, X, H)    
    stdHH = np.sqrt(np.diag(HodrickV.forward(H))) 
    tHH_Tail, tHH_regressor = np.round((rgmodel.params / stdHH)['Tail'], 2), np.round((rgmodel.params / stdHH)[f'{regressor}'], 2)
    TailStocksRET1Y_df = TailStocksRET1Y_df.append({'Rz.':f'{regressor}', 'Tail Coeff.':coeff_Tail, 'Tail t-stat.':tHH_Tail, 'Coeff.':coeff_regressor, 't-stat':tHH_regressor, 'R2':rsquared},ignore_index=True)
print(TailStocksRET1Y_df)

# 3Y 
args.Lwindow = 36
fitdataset3Y = pd.DataFrame()
fitdataset3Y['date'] = fitdataset['date'][:-args.Lwindow]
fitdataset3Y['vwretd'] = cal_aggHorizon_ret(fitdataset['vwretd'], args.Lwindow)   
fitdataset3Y['Tail'] = fitdataset['Tail'][:-args.Lwindow]
for col in regressors:
    fitdataset3Y[col] = fitdataset[col][:-args.Lwindow]
TailStocksRET3Y_df = pd.DataFrame(columns=['Rz.', 'Tail Coeff.', 'Tail t-stat.', 'Coeff.', 't-stat', 'R2'])
for regressor in regressors:
    fitY = fitdataset3Y['vwretd']
    fitX = pd.concat([fitdataset3Y['Tail'], fitdataset3Y[regressor]], axis=1)
    fitX = sm.add_constant(fitX)
    rgmodel = sm.OLS(fitY, fitX).fit()
    coeff_Tail, coeff_regressor = np.round(rgmodel.params['Tail'] / 3 * 100, 2), np.round(rgmodel.params[f'{regressor}'] / 3 * 100, 2)
    tstat_Tail, tstat_regressor = np.round(rgmodel.tvalues['Tail'], 2), np.round(rgmodel.tvalues[f'{regressor}'], 2)
    rsquared = np.round(rgmodel.rsquared * 100, 1)
    # Hodrick standard error
    Ysub_H = np.array(fitY); Ysub_1 = np.array(fitdataset1M['vwretd']); X = np.array(fitX); H = args.Lwindow 
    HodrickV = computeHodrick1992VCovMatrix(Ysub_H, Ysub_1, X, H)    
    stdHH = np.sqrt(np.diag(HodrickV.forward(H))) 
    tHH_Tail, tHH_regressor = np.round((rgmodel.params / stdHH)['Tail'], 2), np.round((rgmodel.params / stdHH)[f'{regressor}'], 2)
    TailStocksRET3Y_df = TailStocksRET3Y_df.append({'Rz.':f'{regressor}', 'Tail Coeff.':coeff_Tail, 'Tail t-stat.':tHH_Tail, 'Coeff.':coeff_regressor, 't-stat':tHH_regressor, 'R2':rsquared}, ignore_index=True)
print(TailStocksRET3Y_df)

# 5Y 
args.Lwindow = 60
fitdataset5Y = pd.DataFrame()
fitdataset5Y['date'] = fitdataset['date'][:-args.Lwindow]
fitdataset5Y['vwretd'] = cal_aggHorizon_ret(fitdataset['vwretd'], args.Lwindow)   
fitdataset5Y['Tail'] = fitdataset['Tail'][:-args.Lwindow]
for col in regressors:
    fitdataset5Y[col] = fitdataset[col][:-args.Lwindow]
TailStocksRET5Y_df = pd.DataFrame(columns=['Rz.', 'Tail Coeff.', 'Tail t-stat.', 'Coeff.', 't-stat', 'R2'])
for regressor in regressors:
    fitY = fitdataset5Y['vwretd']
    fitX = pd.concat([fitdataset5Y['Tail'], fitdataset5Y[regressor]], axis=1)
    fitX = sm.add_constant(fitX)
    rgmodel = sm.OLS(fitY, fitX).fit()
    coeff_Tail, coeff_regressor = np.round(rgmodel.params['Tail'] / 5 * 100, 2), np.round(rgmodel.params[f'{regressor}'] / 5 * 100, 2)
    tstat_Tail, tstat_regressor = np.round(rgmodel.tvalues['Tail'], 2), np.round(rgmodel.tvalues[f'{regressor}'], 2)
    rsquared = np.round(rgmodel.rsquared * 100, 1)
    # Hodrick standard error
    Ysub_H = np.array(fitY); Ysub_1 = np.array(fitdataset1M['vwretd']); X = np.array(fitX); H = args.Lwindow 
    HodrickV = computeHodrick1992VCovMatrix(Ysub_H, Ysub_1, X, H)    
    stdHH = np.sqrt(np.diag(HodrickV.forward(H))) 
    tHH_Tail, tHH_regressor = np.round((rgmodel.params / stdHH)['Tail'], 2), np.round((rgmodel.params / stdHH)[f'{regressor}'], 2)
    TailStocksRET5Y_df = TailStocksRET5Y_df.append({'Rz.':f'{regressor}', 'Tail Coeff.':coeff_Tail, 'Tail t-stat.':tHH_Tail, 'Coeff.':coeff_regressor, 't-stat':tHH_regressor, 'R2':rsquared},ignore_index=True)
print(TailStocksRET5Y_df)
TailStocksRet_df = pd.concat([TailStocksRET1M_df, TailStocksRET1Y_df.iloc[:,1:], TailStocksRET3Y_df.iloc[:,1:], TailStocksRET5Y_df.iloc[:,1:]], axis=1)
args.save_dir = './logs/BivariatePred/'
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)
TailStocksRet_df.to_csv(os.path.join(args.save_dir, f'HillBivariatePred_{args.dataset_start}_{args.dataset_end}.csv'), index=False)

                      Rz.  Tail Coeff.  Tail t-stat.  Coeff.  t-stat   R2
0          Book-to-market         3.31          1.66    2.62    1.31  0.7
1   Default return spread         3.40          1.70    1.55    0.77  0.5
2    Default yield spread         3.31          1.66    3.75    1.88  0.9
3   Dividend payout ratio         3.61          1.81    2.09    1.05  0.6
4    Dividend price ratio         2.82          1.39    3.66    1.80  0.9
5    Earnings price ratio         3.15          1.55    1.92    0.94  0.6
6               Inflation         3.38          1.70   -2.61   -1.31  0.7
7        Long-term return         3.13          1.57    4.99    2.50  1.3
8         Long-term yield         3.79          1.79   -0.71   -0.33  0.5
9    Net equity expansion         3.49          1.72   -0.34   -0.17  0.4
10       Stock volatility         3.41          1.66   -0.58   -0.28  0.5
11            Term spread         3.24          1.57    1.23    0.60  0.5
12     Treasury-bill rate         3.75